In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [2]:
import os
import subprocess
import argparse
from pathlib import Path
import pdb
import warnings

In [3]:
"/scratch/08834/tg881334/CHA_preproc/2.BIDS_data/"

'/scratch/08834/tg881334/CHA_preproc/2.BIDS_data/'

In [4]:
#######ASSUMPTIONS######
curr_dir = os.getcwd()
class Args():
    def __init__(self, BIDS_folder,alloc = "skx-normal"):
        self.alloc = alloc #also possible skx-normal#what to be allocated with 
        self.BIDS_folder = BIDS_folder#'age_4.5_to_5'
        self.base_dir = Path(curr_dir)
        self.scratch_dir = Path('/scratch/08834/tg881334/CHA_preproc') #where the BIDS and outputs are and will be saved
        self.BIDS_dir = Path(self.scratch_dir/'2.BIDS_data/{}'.format(self.BIDS_folder)) #MUST BE MODIFIED TO THE INPUT I WANT TO RUN WITH
        self.save_dir = Path(self.scratch_dir/'3.qsipreproc_results_ADULTS/{}'.format(self.BIDS_folder))
        self.supp_dir = Path('/work2/08834/tg881334/stampede2/CHA_preproc/CHA_preproc_supplementary_files') #directory where the singularity images and etc are held
        #self.shell_dir = Path(self.base_dir / 'step4_shell_outputs/{}/'.format(self.BIDS_folder)) #where the shelll scripts for each subject will be saved
        #self.log_dir = Path(self.base_dir/'step4_shell_outputs/{}/'.format(self.BIDS_folder))
        self.sub_list = sorted(os.listdir(self.BIDS_dir))
        #age 에 관한 것이 있어야 할듯! (특히, freesurfer infant 할떄)
        ages = self.BIDS_folder.split('_')
        print(ages)
        if float(ages[3]) <= 4.5:
            self.infant = True
        elif float(ages[1]) >= 4.5:
            self.infant = False
        else : 
            raise ValueError("something is werid!!! ")
        print(f"infant ? : {self.infant}")

In [5]:
class qsiprep_shell():
    """
    shell script for performing preproc or reconstruction for infants AND adults given their args    
    """
    
    def __init__(self, args, operation ):
        """
        args : 앞에서 정의했어야 했는 것
        perform : what mode to perform, out of ['preproc', 'recon'] 
            * adult preproc : use 0.14.3, full T1 and dwi
            * infant preproc : use 0.14.3, dwi only 
            * adult_recon, infant_recon : not done yet (use 0.16.1)
        """
        if operation == "preproc":
            if args.infant == True:
                self.perform = "infant_preproc"
            else:
                self.perform = "adult_preproc"
        elif operation == "recon":
            raise NotImplementedError("reconstruction not implemented yet")
        else :
            raise ValueError("perform is not either 'preproc' or 'recon'")
        
        #공통된 것들 (preproc이든 recon이든 동일한 것 적용)
        time_lim = {"skx-normal": 48, "skx-dev" : 2}
        
        self.shell_dir = f"{str(args.base_dir)}/step4_shell_output/{self.perform}/{args.BIDS_folder}"#where shell scripts and output/errors will be saved
        os.makedirs(self.shell_dir, exist_ok=True)        
        self.bash_shell = ["#! /bin/bash",
                     f"#SBATCH -J {self.perform}_{args.BIDS_folder}", f"#SBATCH -p {args.alloc}",
                     f"#SBATCH -o {self.shell_dir}/output.%j",
                     f"#SBATCH -e {self.shell_dir}/error.%j",
                     f"#SBATCH -t {time_lim[args.alloc]}:00:00", "#SBATCH -n 1", "#SBATCH -N 1",
                     "#SBATCH --mail-user=dyhan0316@gmail.com", "#SBATCH --mail-type=ALL",
                     "#SBATCH -c 96", "module load tacc-singularity"]
        
        self.args = args #저장하기, so that 밑에 methods에서 쓸 수 있도록
    
    def make_sh_files(self):
        """
        creates sh files for execution
        """
        args = self.args
        
        ##perform mode에 따라서 다르게 학시 
        perform = self.perform #bad coding style, but whatever..ㅋㅋㅋ
        if perform in ['adult_preproc', 'infant_preproc'] : 
            print(f"will perform preprocessing for folder {args.BIDS_folder}")
            #updating bash_shell with the singularity command
            self.sing_img_dir = args.supp_dir/"qsiprep-0.14.3.sif"#singularity image directory (0.14.3.sif for this one )
            os.makedirs(args.save_dir, exist_ok = False)
            self.singularity_command = f"singularity run --cleanenv -B {args.BIDS_dir}/:/data:ro,{args.save_dir}/:/out,{str(args.supp_dir)}:/freesurfer {str(self.sing_img_dir)} /data /out participant -w /out/tmp --output_resolution 1.2 --denoise_after_combining --unringing_method mrdegibbs --b0_to_t1w_transform Affine --intramodal_template_transform SyN --do_reconall --fs-license-file /freesurfer/license.txt --skip_bids_validation --omp-nthreads 4"
            
            #밑에 infant할때 --infant이런것들 넣었는데, --dwi-only도 넣어야할듯?
            #(조금 복잡해짐... 일단은... 두고보자 (그리고 args.infant 를 받도록 했는데, 이것은 좋은 것이 아닐듯..
            #왜냐하면 args.infant = True인데 adult_preproc하라고 내가 말하면 애러가 뜰 것이니 ))
            #below : add dwi-only and --infant if infant 
            if args.infant == True : self.singularity_command = self.singularity_command + ' --infant' + ' --dwi-only' #add infant option of doing infant 
                
            #밑에 warning : 지우기 (나중에 freesurfer도 하고 나면)
            if args.infant == True:
                warnings.warn('infant is half implemented! must do the fressurefr infant, THAN add them up!')      
            
            self.bash_shell.append(self.singularity_command)
            
            #saving the shell files
            
            
            self.shell_shell = f"{self.shell_dir}/run_{perform}_{args.BIDS_folder}.sh"
            with open(self.shell_shell, "x") as f: #"x" so that creates only if it didn't exist
                for line in self.bash_shell:
                    f.write(f"{line}\n")
            
        elif perform in ['adult_recon','infant_recon'] :
            raise NotImplementedError("reconstruction not implemented yet")
            self.sing_img_dir = args.supp_dir/"qsiprep-0.16.1.sif"
        else:
            raise ValueError(f"incorrect perform option : {perform}")
            
        #assert self.infant == False , "NOT AN INFANT! (or infant range is not set correctly in args)"
    
    def run(self):
        """
        runs the created sh files using sbatch 
        """
        args = self.args
        
        
        subprocess.run(f"chmod 755 {self.shell_shell}", shell = True)
        
        subprocess.run(f"sbatch {self.shell_shell}", shell = True)
        
        

#### 하나에 대해서 돌리기 

In [6]:
##실제로 돌리기 

#args = Args(BIDS_folder= "age_7_to_9", alloc = 'skx-dev')
#
#running_it = qsiprep_shell(args, "adult_preproc")
#
#running_it.make_sh_files()

In [7]:
#SBATCH돌리는 것 (login node에서 해야함)
#running_it.run()
#must run sbatch within login => therefore, will have to do this later 

##### 여러개 이제 돌리기 

In [1]:
import os 
for BIDS_folder in os.listdir("/scratch/08834/tg881334/CHA_preproc/2.BIDS_data/"):
    args = Args(BIDS_folder= BIDS_folder, alloc = 'skx-dev') #make args for each BIDS_folder
    
    running_it = qsiprep_shell(args, operation = "preproc")
    
    running_it.make_sh_files()
    running_it.run() #have to be run inside of the login node

NameError: name 'Args' is not defined

In [9]:

num_subs = 0
for age_range in age_list: #age_list를 추가하기 
    print("age_range") #나중에 하기 

NameError: name 'age_list' is not defined

In [ ]:
args.BIDS_dir

In [ ]:
print(args.infant)

In [ ]:
shell_folder = "./step4_qsiprep_shells"
#os.mkdir(shell_folder)

In [ ]:
with open(f"{shell_folder})

In [ ]:
with open("TEST_step4.sh", 'r') as f:
    a = f.read()
print(a)
    

In [ ]:
a[:30]

## 밑 : infant = True일때 하는 것들 (seperate py file 로 만들까?)

In [ ]:
# -*- coding: utf-8 -*- 
"""1. SETUP"""
import os
from REAL_step0_get_infant_adult_df import get_info, get_year_file
import pandas as pd
import shutil
import os
import subprocess

#get info templates
infant_pd, adult_pd = get_info()()

##SETUP THINGS
MRI_path = "/work2/08834/tg881334/stampede2/CHA_preproc/CHA_data_unzip/2010-2015/dicom_2010_2015"
age_list = [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 7, 8, 9, 10, 13, 16, 18,20]

##dataframe concat
infant_df_list = []
adult_df_list = []
for year in infant_pd.keys():
    infant_df_list.append(infant_pd[year].rename(columns = {"Unnamed: 0" : "파일 유무"})) #Unnamed : 0 으로 되어있는 것들을 파일유무라는 것으로 rename을 다시하기
    
for year in adult_pd.keys():
    adult_df_list.append(adult_pd[year].rename(columns = {"Unnamed: 0" : "파일 유무"})) #Unnamed : 0 으로 되어있는 것들을 파일유무라는 것으로 rename을 다시하기

#concat the things
infant_df_total = pd.concat([i for i in infant_df_list])
adult_df_total = pd.concat([i for i in adult_df_list])
total_df = pd.concat([infant_df_total, adult_df_total])


In [ ]:
total_df

In [ ]:
sub_list = [i for i in os.listdir(args.BIDS_dir) if i !="dataset_description.json"] #remove dataset_description

In [ ]:
len(sub_list)

In [ ]:
[:]